# Multiword expression identification and extraction
Mateusz Wojtulewicz

In [1]:
import spacy
import tqdm

from pathlib import Path

## 1. Use SpaCy tokenizer API to tokenize the text from the law corpus.

In [2]:
nlp = spacy.load("pl_core_news_sm")
tokenizer = nlp.tokenizer

In [3]:
tokenized_acts = {}

acts_dir = Path("../data/ustawy/")
n_acts = len(list(acts_dir.iterdir()))

for act in tqdm.tqdm(acts_dir.iterdir(), desc="Tokenizing acts", total=n_acts):
    act_id = act.stem
    content = act.read_text(encoding="utf8")
    tokens = tokenizer(content)
    tokenized_acts[act_id] = [token.text.lower() for token in tokens]

Tokenizing acts: 100%|██████████| 1179/1179 [00:37<00:00, 31.82it/s]


## 2. Compute bigram counts of downcased tokens.

In [4]:
from collections import Counter

In [5]:
bigrams = (
    (f, s)
    for tokens in tokenized_acts.values()
    for (f, s) in zip(tokens[:-1], tokens[1:])
)

In [6]:
bigrams_count = Counter(bigrams)

In [7]:
bigrams_count.most_common(10)

[(('art', '.'), 83778),
 (('ust', '.'), 53552),
 (('.', '\n'), 49741),
 (('poz', '.'), 45198),
 ((',', 'poz'), 39655),
 (('-', '-'), 36542),
 (('r', '.'), 33015),
 (('w', 'art'), 30170),
 (('.', '1'), 29734),
 ((',', 'o'), 28739)]

## 3. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.

I'm using a Token's property `is_alpha` which is `True` if the token consists only of alphabetic characters.

In [8]:
def is_word(s: str) -> bool:
   return tokenizer(s)[0].is_alpha 

In [9]:
bigrams_filtered = (
    (f, s)
    for tokens in tokenized_acts.values()
    for (f, s) in zip(tokens[:-1], tokens[1:])
    if is_word(f) and is_word(s)
)

In [10]:
bigrams_filtered_count = Counter(bigrams_filtered)

In [11]:
bigrams_filtered_count.most_common(20)

[(('w', 'art'), 30170),
 (('mowa', 'w'), 27649),
 (('w', 'ust'), 22238),
 (('których', 'mowa'), 12973),
 (('o', 'których'), 12604),
 (('otrzymuje', 'brzmienie'), 9168),
 (('z', 'dnia'), 8989),
 (('którym', 'mowa'), 8689),
 (('o', 'którym'), 8525),
 (('do', 'spraw'), 8215),
 (('dodaje', 'się'), 7960),
 (('i', 'nr'), 7871),
 (('w', 'brzmieniu'), 6690),
 (('w', 'drodze'), 6623),
 (('stosuje', 'się'), 6232),
 (('na', 'podstawie'), 5906),
 (('w', 'przypadku'), 5371),
 (('której', 'mowa'), 5192),
 (('o', 'której'), 5061),
 (('od', 'dnia'), 4971)]

## 4. Use pointwise mutual information to compute the measure for all pairs of words.
The _PMI_ of a pair of outcomes $x$ and $y$ is calculated as follows:
$$
\text{pmi}(x; y) = \log_2 \frac{p(x, y)}{p(x)p(y)}
$$

In our case:
- $p(x, y)$ is a probability of drawing a bigram $(x,y)$ from all bigrams in the corpus;
- $p(x)$ is a probabilty of drawing a token $x$ from all tokens in the corpus.

In [12]:
tokens_count = Counter(
    token for tokens in tokenized_acts.values() for token in tokens
)

In [13]:
tokens_count.most_common(10)

[('.', 437694),
 (',', 341126),
 ('w', 201224),
 ('\n', 181703),
 (')', 100194),
 ('i', 90009),
 ('art', 83804),
 ('z', 82443),
 ('1', 73108),
 ('o', 64776)]

Saving counters to be used to calculate _PMI_ in a `pmi_bigram_mp.py` script that uses parallelization.

In [16]:
import pickle

def dump_counter(counter: Counter, name: str) -> None:
    with open(f"{name}.p", "wb") as f:
        pickle.dump(counter, f)

In [17]:
dump_counter(bigrams_count, "bigrams_counter")
dump_counter(bigrams_filtered_count, "bigrams_filtered_counter")
dump_counter(tokens_count, "tokens_counter")

Loading calculated results.

In [20]:
bigrams_pmi = pickle.load(open("pmi_results.p", "rb"))

## 5. Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [23]:
sorted(bigrams_pmi, key=lambda x: -x[1])[:10]

[(('kołowe', 'jednoosiowe'), 22.475522182157892),
 (('zbrojeń', 'żelbeto'), 22.475522182157892),
 (('prefabrykatów', 'wnętrzowe'), 22.475522182157892),
 (('gołe', 'aluminiowe'), 22.475522182157892),
 (('polistyrenu', 'spienionego'), 22.475522182157892),
 (('objaśnieniem', 'figur'), 22.475522182157892),
 (('wkładzie', 'wnoszonym'), 22.475522182157892),
 (('doktorem', 'habilitowanym'), 22.475522182157892),
 (('losy', 'loteryjne'), 22.475522182157892),
 (('ugaszone', 'zapałki'), 22.475522182157892)]

## 6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).

In [28]:
bigrams_pmi_5 = [(bigram, pmi) for bigram, pmi in bigrams_pmi if bigrams_count[bigram] > 5]
sorted(bigrams_pmi_5, key=lambda x: -x[1])[:10]

[(('diagności', 'laboratoryjni'), 19.890559681436738),
 (('odczynów', 'poszczepiennych'), 19.890559681436738),
 (('adama', 'mickiewicza'), 19.890559681436738),
 (('papierem', 'wartościowym'), 19.66816726010029),
 (('piotrków', 'trybunalski'), 19.66816726010029),
 (('lambrekiny', 'okienne'), 19.66816726010029),
 (('schedę', 'spadkową'), 19.66816726010029),
 (('buraka', 'cukrowego'), 19.475522182157892),
 (('zdrowego', 'stylu'), 19.475522182157892),
 (('zapieczętowanej', 'kopercie'), 19.445774838763842)]

## 7. Use KRNNT or Clarin-PL API to tag and lemmatize the corpus.

I've used Clarin-PL API with Moreusz 1, and then I've downloaded the XML output files.

In [29]:
import xml.etree.ElementTree as ET

In [86]:
tag_tokenized_acts = {}

tag_acts_dir = Path("../data/ustawy_tagged_clarin_morf1/")
n_acts = len(list(tag_acts_dir.iterdir()))

for act in tqdm.tqdm(tag_acts_dir.iterdir(), desc="Tokenizing tagged acts", total=n_acts):
    act_id = act.stem
    content = act.read_text(encoding="utf8")
    tree = ET.fromstring(text=content)
    tag_tokens = []
    for tok in tree.iter("tok"):
        lex = tok.find("lex")
        base = lex.find("base").text
        ctag = lex.find("ctag").text
        tag_token = f"{base.lower()}:{ctag.split(':')[0]}"
        tag_tokens.append(tag_token)
    tag_tokenized_acts[act_id] = tag_tokens

Tokenizing tagged acts: 100%|██████████| 1179/1179 [02:16<00:00,  8.63it/s]


## 8. Using the tagged corpus compute bigram statistic for the tokens containing: a. lemmatized, downcased word b. morphosyntactic category of the word (subst, fin, adj, etc.)

In [87]:
tag_bigrams = [
    (f, s)
    for tag_tokens in tag_tokenized_acts.values()
    for (f, s) in zip(tag_tokens[:-1], tag_tokens[1:])
]

In [88]:
tag_bigrams_counter = Counter(tag_bigrams)

In [89]:
tag_bigrams_counter.most_common(10)

[(('art:brev', '.:interp'), 83700),
 (('usta:subst', '.:interp'), 53553),
 (('poz:brev', '.:interp'), 45195),
 ((',:interp', 'poz:brev'), 43166),
 (('.:interp', '1:adj'), 39987),
 (('-:interp', '-:interp'), 36548),
 (('r:brev', '.:interp'), 33029),
 (('w:prep', 'art:brev'), 31988),
 ((',:interp', 'o:prep'), 29906),
 (('o:prep', 'który:adj'), 28646)]

Now, I am filtering tokens that base is a word.

In [95]:
def is_word_base(token: str) -> bool:
    parts = token.split(":")
    if len(parts) != 2:
        return False
    else:
        return is_word(s=parts[0])

In [96]:
tag_bigrams_filtered = (
    (f, s)
    for tag_tokens in tag_tokenized_acts.values()
    for (f, s) in zip(tag_tokens[:-1], tag_tokens[1:])
    if is_word_base(f) and is_word_base(s)
)

In [97]:
tag_bigrams_filtered_counter = Counter(tag_bigrams_filtered)

In [99]:
tag_bigrams_filtered_counter.most_common(20)

[(('w:prep', 'art:brev'), 31988),
 (('o:prep', 'który:adj'), 28646),
 (('który:adj', 'mowa:subst'), 28538),
 (('mowa:subst', 'w:prep'), 28473),
 (('w:prep', 'usta:subst'), 23557),
 (('z:prep', 'dzień:subst'), 11360),
 (('otrzymywać:fin', 'brzmienie:subst'), 10536),
 (('określić:ppas', 'w:prep'), 10169),
 (('do:prep', 'sprawa:subst'), 8716),
 (('ustawa:subst', 'z:prep'), 8625),
 (('właściwy:adj', 'do:prep'), 8535),
 (('i:conj', 'nr:brev'), 8435),
 (('dodawać:fin', 'się:qub'), 8195),
 (('minister:subst', 'właściwy:adj'), 7931),
 (('w:prep', 'brzmienie:subst'), 7280),
 (('w:prep', 'droga:subst'), 7128),
 (('w:prep', 'przypadek:subst'), 6776),
 (('na:prep', 'podstawa:subst'), 6681),
 (('stosować:fin', 'się:qub'), 6535),
 (('się:qub', 'wyraz:subst'), 6077)]

## 10. Compute the same statistics as for the non-lemmatized words (i.e. PMI) and print top-10 entries with at least 5 occurrences.

In [100]:
tag_tokens_counter = Counter(
    tag_token for tag_tokens in tag_tokenized_acts.values() for tag_token in tag_tokens
)

In [101]:
tag_tokens_counter.most_common(10)

[('.:interp', 457356),
 (',:interp', 343058),
 ('w:prep', 202567),
 ('):interp', 102195),
 ('z:prep', 87991),
 ('i:conj', 87725),
 ('art:brev', 83706),
 ('1:adj', 74500),
 ('o:prep', 64712),
 ('-:interp', 61833)]

Saving counters to be used to calculate _PMI_ in a `pmi_bigram_mp.py` script that uses parallelization.

In [103]:
dump_counter(tag_bigrams_counter, "tag_bigrams_counter")
dump_counter(tag_bigrams_filtered_counter, "tag_bigrams_filtered_counter")
dump_counter(tag_tokens_counter, "tag_tokens_counter")

Loading calculated results.

In [104]:
tag_bigrams_pmi = pickle.load(open("tag_pmi_results.p", "rb"))

Top 10 bigrams with any number of occurrences.

In [105]:
sorted(tag_bigrams_pmi, key=lambda x: -x[1])[:10]

[(('tornister:subst', 'nieskórzany:adj'), 22.330150430961186),
 (('zbrojenia:subst', 'żelbeto:adja'), 22.330150430961186),
 (('reduktor:subst', 'membranowy:adj'), 22.330150430961186),
 (('prefabrykat:subst', 'wnętrzowy:adj'), 22.330150430961186),
 (('polistyren:subst', 'spienić:ppas'), 22.330150430961186),
 (('uw:subst', 'zględnieniu:subst'), 22.330150430961186),
 (('english:subst', 'language:subst'), 22.330150430961186),
 (('language:subst', 'college:subst'), 22.330150430961186),
 (('southern:subst', 'trade:subst'), 22.330150430961186),
 (('łęka:subst', 'dukielski:adj'), 22.330150430961186)]

Top 10 bigrams with at least 5 occurrences.

In [106]:
tag_bigrams_pmi_5 = [(tag_bigram, pmi) for tag_bigram, pmi in tag_bigrams_pmi if tag_bigrams_counter[tag_bigram] > 5]
sorted(tag_bigrams_pmi_5, key=lambda x: -x[1])[:10]

[(('adam:subst', 'mickiewicz:subst'), 19.74518793024003),
 (('piotrków:subst', 'trybunalski:adj'), 19.52279550890358),
 (('przeponowy:adj', 'rurowy:adj'), 19.330150430961186),
 (('chrześcijanin:subst', 'baptysta:subst'), 19.10775800962474),
 (('maria:subst', 'curie:subst'), 19.00822233607382),
 (('hugo:subst', 'kołłątaj:subst'), 19.00822233607382),
 (('tadeusz:subst', 'kotarbiński:subst'), 19.00822233607382),
 (('upośledzony:adj', 'umysłowo:adv'), 18.870718812323886),
 (('jedwab:subst', 'wiskozowy:adj'), 18.870718812323886),
 (('vitis:subst', 'vinifera:subst'), 18.870718812323886)]

## 11. Compute **trigram** counts for both corpora and perform the same filtering.

### A. Non-lemmatized corpora.

In [107]:
trigrams_counter = Counter(
    (first, second, third)
    for tokens in tokenized_acts.values()
    for (first, second, third) in zip(tokens, tokens[1:], tokens[2:])
)

In [108]:
trigrams_counter.most_common(10)

[((',', 'poz', '.'), 39633),
 (('-', '-', '-'), 34641),
 (('w', 'art', '.'), 30162),
 (('ust', '.', '1'), 22618),
 (('w', 'ust', '.'), 22204),
 (('r', '.', 'nr'), 16956),
 (('_', '_', '_'), 16111),
 (('których', 'mowa', 'w'), 12506),
 (('mowa', 'w', 'ust'), 12388),
 ((',', 'o', 'których'), 12009)]

In [110]:
trigrams_filtered_counter = Counter(
    (first, second, third)
    for tokens in tokenized_acts.values()
    for (first, second, third) in zip(tokens, tokens[1:], tokens[2:])
    if is_word(first) and is_word(second) and is_word(third)
)

In [111]:
trigrams_filtered_counter.most_common(10)

[(('których', 'mowa', 'w'), 12506),
 (('mowa', 'w', 'ust'), 12388),
 (('o', 'których', 'mowa'), 11714),
 (('mowa', 'w', 'art'), 10995),
 (('którym', 'mowa', 'w'), 8429),
 (('o', 'którym', 'mowa'), 8040),
 (('której', 'mowa', 'w'), 5020),
 (('o', 'której', 'mowa'), 4731),
 (('właściwy', 'do', 'spraw'), 4434),
 (('minister', 'właściwy', 'do'), 4106)]

### B. Lemmatized and tagged corpora.

In [112]:
tag_trigrams_counter = Counter(
    (first, second, third)
    for tokens in tag_tokenized_acts.values()
    for (first, second, third) in zip(tokens, tokens[1:], tokens[2:])
)

In [113]:
tag_trigrams_counter.most_common(10)

[((',:interp', 'poz:brev', '.:interp'), 43165),
 (('-:interp', '-:interp', '-:interp'), 34646),
 (('w:prep', 'art:brev', '.:interp'), 31987),
 (('o:prep', 'który:adj', 'mowa:subst'), 28525),
 ((',:interp', 'o:prep', 'który:adj'), 28445),
 (('który:adj', 'mowa:subst', 'w:prep'), 28442),
 (('w:prep', 'usta:subst', '.:interp'), 23520),
 (('usta:subst', '.:interp', '1:adj'), 23346),
 (('.:interp', 'art:brev', '.:interp'), 22917),
 (('r:brev', '.:interp', 'nr:brev'), 17860)]

In [114]:
tag_trigrams_filtered_counter = Counter(
    (first, second, third)
    for tokens in tag_tokenized_acts.values()
    for (first, second, third) in zip(tokens, tokens[1:], tokens[2:])
    if is_word(first) and is_word(second) and is_word(third)
)

In [115]:
tag_trigrams_filtered_counter.most_common(10)

[(('o:prep', 'który:adj', 'mowa:subst'), 28525),
 (('który:adj', 'mowa:subst', 'w:prep'), 28442),
 (('mowa:subst', 'w:prep', 'usta:subst'), 13474),
 (('mowa:subst', 'w:prep', 'art:brev'), 12293),
 (('ustawa:subst', 'z:prep', 'dzień:subst'), 8589),
 (('właściwy:adj', 'do:prep', 'sprawa:subst'), 7964),
 (('minister:subst', 'właściwy:adj', 'do:prep'), 7886),
 (('w:prep', 'droga:subst', 'rozporządzenie:subst'), 4748),
 (('zastępować:fin', 'się:qub', 'wyraz:subst'), 3653),
 (('w:prep', 'ustawa:subst', 'z:prep'), 3646)]

## 12. Use PMI (with 5 occurrence threshold) to compute top 10 results for the trigrams. Devise a method for computing the values, based on the results for bigrams.

I am constructing _PMI_ of three outcomes $x$, $y$, $z$ to be:
$$
\text{pmi}(x; y; z) = \log_2 \frac{p(x, y, z)}{p(x)p(y)p(z)}
$$
For this constructions still holds that _PMI_ of outcomes measures the divergence between probability of their coincidence and the probability of their individual, independent occurences.

Saving counters to be used to calculate _PMI_ in a `pmi_trigram_mp.py` script that uses parallelization.

In [117]:
dump_counter(trigrams_counter, "trigrams_counter")
dump_counter(trigrams_filtered_counter, "trigrams_filtered_counter")

dump_counter(tag_trigrams_counter, "tag_trigrams_counter")
dump_counter(tag_trigrams_filtered_counter, "tag_trigrams_filtered_counter")

Loading calculated results.